In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
import numpy as np
import pandas as pd

Species_df = pd.read_csv('Species_data.csv')

Species_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

y = Species_df['category']
X = Species_df.drop(columns='category')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=84)

scaler = MinMaxScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled)

In [ ]:
neural = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(50, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural.summary()

In [ ]:
neural.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [ ]:
training = neural.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
loss, accuracy = neural.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

In [ ]:
# def create_model(num_hidden_layers=1,layer_size=32,activation='relu', optimizer='adam', learning_rate=0.001):
#     model = tf.keras.Sequential()
#     model.add(layers.Input(shape=(X_train_scaled.shape[1],)))
#     for _ in range(num_hidden_layers):
#         model.add(layers.Dense(layer_size, activation=activation))
#     model.add(layers.Dense(3, activation='softmax'))

#     opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#     model.compile(optimizer='adam', loss='spare_classification_crossentropy', metrics=['accuracy', 'loss'])

    

In [ ]:
# if hasattr(tf.keras, 'wrappers'):
#     print(dir(tf.keras.wrappers))
#     if hasattr(tf.keras.wrappers, 'scikit_learn'):
#         print(dir(tf.keras.wrappers.scikit_learn))

In [ ]:
# from tensorflow.keras.wrappers import SKLearnClassifier


In [ ]:
# from tensorflow.keras import layers
# from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier

# keras_clf = SKLearnClassifier(model=create_model)

In [ ]:
param_grid = {
    'num_hidden_layers': [1, 2],
    'layer_size': [70, 128],
    'activation': ['relu', 'tanh'],
    'optimizer': ['adam'],
    'learning_rate': [0.01, 0.001],
    'epochs': [5, 10],
    'batch_size': [32, 64]
}

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.neural_network import MLPClassifier

# grid_search = GridSearchCV(estimator=neural, param_grid=param_grid, scoring='accuracy', cv=3)


In [ ]:
# grid_search.fit(X_train_scaled, y_train)

In [ ]:
neural2 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(50, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(15, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural2.summary()

neural2.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural2.fit(X_train_scaled, y_train, epochs=40, batch_size=32, validation_split=0.2)


In [ ]:
neural3 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(50, activation='relu'),
    layers.Dense(35, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(15, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural3.summary()

neural3.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural3.fit(X_train_scaled, y_train, epochs=40, batch_size=32, validation_split=0.2)

In [ ]:
neural4 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(50, activation='relu'),
    layers.Dense(35, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(15, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural4.summary()

neural4.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural4.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

In [ ]:
neural5 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(50, activation='relu'),
    layers.Dense(35, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(15, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural5.summary()

neural5.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural5.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
neural6 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(50, activation='relu'),
    layers.Dense(35, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(30, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(15, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural6.summary()

neural6.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural6.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
neural7 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(50, activation='relu'),
    layers.Dense(42, activation='relu'),
    layers.Dense(36, activation='relu'),
    layers.Dense(30, activation='relu'),
    layers.Dense(27, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(15, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural7.summary()

neural7.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural7.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.2)

In [ ]:
loss, accuracy = neural7.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

In [ ]:
neural8 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(42, activation='relu'),
    layers.Dense(35, activation='relu'),
    layers.Dense(27, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(14, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural8.summary()

neural8.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural8.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.2)

In [ ]:
loss, accuracy = neural8.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

In [ ]:
loss, accuracy = neural3.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

In [ ]:
loss, accuracy = neural4.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

In [ ]:
loss, accuracy = neural5.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

In [ ]:
loss, accuracy = neural6.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

In [ ]:
loss, accuracy = neural7.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

In [ ]:
neural9 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(42, activation='relu'),
    layers.Dense(35, activation='tanh'),
    layers.Dense(27, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(14, activation='relu'),
    layers.Dense(9, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural9.summary()

neural9.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural9.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.2)

In [ ]:
loss, accuracy = neural9.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

In [ ]:
neural10 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(45, activation='relu'),
    layers.Dense(35, activation='tanh'),
    layers.Dense(25, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(7, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural10.summary()

neural10.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural10.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.3)

In [ ]:
loss, accuracy = neural10.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

In [ ]:
neural10 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(45, activation='relu'),
    layers.Dense(35, activation='tanh'),
    layers.Dense(25, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(7, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural10.summary()

neural10.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural10.fit(X_train_scaled, y_train, epochs=400, batch_size=32, validation_split=0.3)

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_148 (Dense)               │ (None, 70)             │         2,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_149 (Dense)               │ (None, 45)             │         3,195 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_150 (Dense)               │ (None, 35)             │         1,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_151 (Dense)               │ (None, 25)             │           900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_152 (Dense)               │ (None, 10)             │           260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_153 (Dense)               │ (None, 7)              │            77 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_154 (Dense)               │ (None, 3)              │            24 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,586 (33.54 KB)

 Trainable params: 8,586 (33.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/400
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4882 - loss: 0.9994 - val_accuracy: 0.6008 - val_loss: 0.8098
Epoch 2/400
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step - accuracy: 0.6319 - loss: 0.7634 - val_accuracy: 0.6611 - val_loss: 0.7033
Epoch 3/400
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - accuracy: 0.6676 - loss: 0.6828 - val_accuracy: 0.6720 - val_loss: 0.6469
Epoch 4/400
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step - accuracy: 0.7021 - loss: 0.6298 - val_accuracy: 0.6927 - val_loss: 0.6440
Epoch 5/400
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - accuracy: 0.7193 - loss: 0.6067 - val_accuracy: 0.7214 - val_loss: 0.6036
Epoch 6/400
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step - accuracy: 0.7229 - loss: 0.5939 - val_accuracy: 0.7128 - val_loss: 0.6059
Epoch 7/400
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accuracy: 0.7417 - loss: 0.5799 - val_accuracy: 0.7455 - val_loss: 0.5804
Epoch 8/400
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.7486 - loss: 0.5

In [101]:
loss, accuracy = neural10.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

78/78 - 0s - 491us/step - accuracy: 0.7632 - loss: 1.1793
Test Loss: 1.179
Test Accuracy: 0.763


In [ ]:
# print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


In [102]:
neural11 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(45, activation='relu'),
    layers.Dense(35, activation='tanh'),
    layers.Dense(25, activation='relu'),
    layers.Dense(15, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural11.summary()

neural11.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural11.fit(X_train_scaled, y_train, epochs=250, batch_size=27, validation_split=0.25)

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_155 (Dense)               │ (None, 70)             │         2,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_156 (Dense)               │ (None, 45)             │         3,195 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_157 (Dense)               │ (None, 35)             │         1,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_158 (Dense)               │ (None, 25)             │           900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_159 (Dense)               │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_160 (Dense)               │ (None, 10)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_161 (Dense)               │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,808 (34.41 KB)

 Trainable params: 8,808 (34.41 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 930us/step - accuracy: 0.5077 - loss: 1.0074 - val_accuracy: 0.5755 - val_loss: 0.8016
Epoch 2/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - accuracy: 0.6071 - loss: 0.7603 - val_accuracy: 0.6306 - val_loss: 0.6990
Epoch 3/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - accuracy: 0.6516 - loss: 0.7118 - val_accuracy: 0.6919 - val_loss: 0.6453
Epoch 4/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - accuracy: 0.7132 - loss: 0.6321 - val_accuracy: 0.7367 - val_loss: 0.6149
Epoch 5/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - accuracy: 0.7390 - loss: 0.6080 - val_accuracy: 0.7119 - val_loss: 0.6236
Epoch 6/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - accuracy: 0.7499 - loss: 0.5683 - val_accuracy: 0.7471 - val_loss: 0.5690
Epoch 7/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - accuracy: 0.7480 - loss: 0.5615 - val_accuracy: 0.7078 - val_loss: 0.5935
Epoch 8/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - accuracy: 0.7478 - loss: 0

In [103]:
loss, accuracy = neural11.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

78/78 - 0s - 498us/step - accuracy: 0.7612 - loss: 0.9112
Test Loss: 0.911
Test Accuracy: 0.761


In [106]:
X_val_scaled = X_test_scaled
y_val = y_test

In [114]:
def build_model(num_hidden_layers, layer_size_list, activation='relu', output_activation='softmax'):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(X_train_scaled.shape[1],)))

    if len(layer_size_list) != num_hidden_layers:
        raise ValueError("Length of layer sizes not equal")
    
    for size in layer_size_list:
        model.add(layers.Dense(size, activation=activation))

    model.add(layers.Dense(3, activation=output_activation))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# hyperparameters

num_hidden_layers_to_test = [1,2,3]
layer_sizes_to_test = [[70], [70, 35], [70, 32, 14], [70, 45, 15], [70, 49, 28]]
activation_functions_to_test = ['relu', 'tanh']

best_val_accuracy = 0.0
best_model_config = None
all_histories = {}




In [115]:
for num_layers in num_hidden_layers_to_test:
    for layer_sizes in layer_sizes_to_test:
        if len(layer_sizes) == num_layers:
            for activation in activation_functions_to_test:
                print(f"\nTesting model with {num_layers} hidden layers, sizes {layer_sizes}, activation {activation}")

                model = build_model(num_layers, layer_sizes, activation)
                history = model.fit(
                    X_train_scaled, y_train,
                    epochs=100,
                    batch_size=35,
                    validation_data=(X_val_scaled, y_val),
                    verbose=0
                )

                val_accuracy = history.history['val_accuracy'][-1]
                print(f'Validation Accuracy: {val_accuracy:.4f}')

                config_str = f"Layers: {num_layers}, Sizes: {layer_sizes}, Activation: {activation}"
                all_histories[config_str] = history.history

                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    best_model_config = {"num_layers": num_layers, "layer_sizes": layer_sizes, "activation": activation}
                    best_model = model

print("\nBest Model Configuration:")
print(f"Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Hyperparameters: {best_model_config}")



Testing model with 1 hidden layers, sizes [70], activation relu
Validation Accuracy: 0.7825

Testing model with 1 hidden layers, sizes [70], activation tanh
Validation Accuracy: 0.7298

Testing model with 2 hidden layers, sizes [70, 35], activation relu
Validation Accuracy: 0.7865

Testing model with 2 hidden layers, sizes [70, 35], activation tanh
Validation Accuracy: 0.7772

Testing model with 3 hidden layers, sizes [70, 32, 14], activation relu
Validation Accuracy: 0.7813

Testing model with 3 hidden layers, sizes [70, 32, 14], activation tanh
Validation Accuracy: 0.7704

Testing model with 3 hidden layers, sizes [70, 45, 15], activation relu
Validation Accuracy: 0.7748

Testing model with 3 hidden layers, sizes [70, 45, 15], activation tanh
Validation Accuracy: 0.7845

Testing model with 3 hidden layers, sizes [70, 49, 28], activation relu
Validation Accuracy: 0.7776

Testing model with 3 hidden layers, sizes [70, 49, 28], activation tanh
Validation Accuracy: 0.7720

Best Model Co

In [113]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


In [117]:
def build_model(num_hidden_layers, layer_size_list, activation='relu', output_activation='softmax'):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(X_train_scaled.shape[1],)))

    if len(layer_size_list) != num_hidden_layers:
        raise ValueError("Length of layer sizes not equal")
    
    for size in layer_size_list:
        model.add(layers.Dense(size, activation=activation))

    model.add(layers.Dense(3, activation=output_activation))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# hyperparameters

num_hidden_layers_to_test = [1,2,3]
layer_sizes_to_test = [[70], [55, 35], [49, 28], [50, 25, 15], [60, 35, 14], [49, 35, 27]]
activation_functions_to_test = ['relu']

best_val_accuracy = 0.0
best_model_config = None
all_histories = {}

for num_layers in num_hidden_layers_to_test:
    for layer_sizes in layer_sizes_to_test:
        if len(layer_sizes) == num_layers:
            for activation in activation_functions_to_test:
                print(f"\nTesting model with {num_layers} hidden layers, sizes {layer_sizes}, activation {activation}")

                model = build_model(num_layers, layer_sizes, activation)
                history = model.fit(
                    X_train_scaled, y_train,
                    epochs=100,
                    batch_size=35,
                    validation_data=(X_val_scaled, y_val),
                    verbose=0
                )

                val_accuracy = history.history['val_accuracy'][-1]
                print(f'Validation Accuracy: {val_accuracy:.4f}')

                config_str = f"Layers: {num_layers}, Sizes: {layer_sizes}, Activation: {activation}"
                all_histories[config_str] = history.history

                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    best_model_config = {"num_layers": num_layers, "layer_sizes": layer_sizes, "activation": activation}
                    best_model = model

print("\nBest Model Configuration:")
print(f"Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Hyperparameters: {best_model_config}")




Testing model with 1 hidden layers, sizes [70], activation relu
Validation Accuracy: 0.7672

Testing model with 2 hidden layers, sizes [55, 35], activation relu
Validation Accuracy: 0.7845

Testing model with 2 hidden layers, sizes [49, 28], activation relu
Validation Accuracy: 0.7784

Testing model with 3 hidden layers, sizes [50, 25, 15], activation relu
Validation Accuracy: 0.8006

Testing model with 3 hidden layers, sizes [60, 35, 14], activation relu
Validation Accuracy: 0.7885

Testing model with 3 hidden layers, sizes [49, 35, 27], activation relu
Validation Accuracy: 0.7692

Best Model Configuration:
Validation Accuracy: 0.8006
Hyperparameters: {'num_layers': 3, 'layer_sizes': [50, 25, 15], 'activation': 'relu'}


In [118]:
neural12 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(50, activation='relu'),
    layers.Dense(25, activation='tanh'),
    layers.Dense(15, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural12.summary()

neural12.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural12.fit(X_train_scaled, y_train, epochs=250, batch_size=27, validation_split=0.2)

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_55"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_267 (Dense)               │ (None, 70)             │         2,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_268 (Dense)               │ (None, 50)             │         3,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_269 (Dense)               │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_270 (Dense)               │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_271 (Dense)               │ (None, 3)              │            48 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,783 (30.40 KB)

 Trainable params: 7,783 (30.40 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/250
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - accuracy: 0.5627 - loss: 0.8771 - val_accuracy: 0.6434 - val_loss: 0.7413
Epoch 2/250
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - accuracy: 0.6479 - loss: 0.6999 - val_accuracy: 0.6606 - val_loss: 0.6637
Epoch 3/250
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - accuracy: 0.6871 - loss: 0.6336 - val_accuracy: 0.7054 - val_loss: 0.6285
Epoch 4/250
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step - accuracy: 0.7151 - loss: 0.6044 - val_accuracy: 0.7330 - val_loss: 0.6149
Epoch 5/250
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - accuracy: 0.7290 - loss: 0.5924 - val_accuracy: 0.7313 - val_loss: 0.5990
Epoch 6/250
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - accuracy: 0.7404 - loss: 0.5673 - val_accuracy: 0.7425 - val_loss: 0.5804
Epoch 7/250
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step - accuracy: 0.7661 - loss: 0.5486 - val_accuracy: 0.7511 - val_loss: 0.5618
Epoch 8/250
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - accuracy: 0.7525 - loss: 0

In [133]:
loss, accuracy = neural12.evaluate(X_val_scaled, y_val, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

78/78 - 0s - 486us/step - accuracy: 0.7728 - loss: 1.0200
Test Loss: 1.020
Test Accuracy: 0.773


In [168]:
neural13 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(35, activation='relu'),
    layers.Dense(21, activation='relu'),
    layers.Dense(14, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural13.summary()

neural13.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural13.fit(X_train_scaled, y_train, epochs=200, batch_size=32, validation_split=0.3)

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_69"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_337 (Dense)               │ (None, 70)             │         2,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_338 (Dense)               │ (None, 35)             │         2,485 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_339 (Dense)               │ (None, 21)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_340 (Dense)               │ (None, 14)             │           308 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_341 (Dense)               │ (None, 3)              │            45 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,114 (23.88 KB)

 Trainable params: 6,114 (23.88 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4692 - loss: 0.9806 - val_accuracy: 0.6077 - val_loss: 0.8180
Epoch 2/200
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - accuracy: 0.6188 - loss: 0.7857 - val_accuracy: 0.6531 - val_loss: 0.7128
Epoch 3/200
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 585us/step - accuracy: 0.6788 - loss: 0.6769 - val_accuracy: 0.7145 - val_loss: 0.6710
Epoch 4/200
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step - accuracy: 0.7107 - loss: 0.6446 - val_accuracy: 0.7157 - val_loss: 0.6269
Epoch 5/200
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step - accuracy: 0.7346 - loss: 0.5961 - val_accuracy: 0.7329 - val_loss: 0.6098
Epoch 6/200
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - accuracy: 0.7367 - loss: 0.5845 - val_accuracy: 0.7128 - val_loss: 0.5941
Epoch 7/200
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step - accuracy: 0.7417 - loss: 0.5700 - val_accuracy: 0.7059 - val_loss: 0.5916
Epoch 8/200
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - accuracy: 0.7525 - loss: 0.5

In [170]:
loss, accuracy = neural13.evaluate(X_val_scaled, y_val, verbose=2)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

78/78 - 0s - 481us/step - accuracy: 0.7764 - loss: 0.7845
Test Loss: 0.784
Test Accuracy: 0.776


Best NN model config: 

neural13 = tf.keras.Sequential([
    layers.Dense(70, activation='relu', input_shape=(35,)),
    layers.Dense(35, activation='relu'),
    layers.Dense(21, activation='relu'),
    layers.Dense(14, activation='relu'),
    layers.Dense(3, activation='softmax')
])

neural13.summary()

neural13.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

training = neural13.fit(X_train_scaled, y_train, epochs=200, batch_size=32, validation_split=0.3)